In [ ]:
# Loading drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
# All general imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer 

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Reshape, Conv2D, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Bidirectional, GlobalAveragePooling1D, GRU, GlobalMaxPooling1D, concatenate
from keras.optimizers import Adam
from keras.layers import LSTM, GRU, Conv1D, MaxPool1D, Activation

from keras.models import Model, Sequential
from keras.layers.core import SpatialDropout1D

from keras.engine.topology import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, Softmax
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import backend as K

from keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import io, os, gc

In [ ]:
# Setting the working directory 
!ls
%cd drive/My\ Drive/Fake_News_Data
!pwd

drive  sample_data
/content/drive/My Drive/Fake_News_Data
/content/drive/My Drive/Fake_News_Data


In [ ]:
go_train = "gotrain_bin.tsv"
go_test = "gotest_bin.tsv"
df_train = pd.read_csv(go_train, sep='\t', header=None)
df_test = pd.read_csv(go_test, sep='\t', header=None)
df_train.columns = ['Text', 'Label', 'ID']
df_test.columns = ['Text', 'Label', 'ID']

In [ ]:
print(df_train.head())
print(df_test.head())

                                                Text  Label       ID
0                     WHY THE FUCK IS BAYLESS ISOING      1  eezlygj
1                        To make her feel threatened      1  ed7ypvh
2  OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...      1  edvnz26
3                                    Fucking coward.      1  edk0z9k
4              Stupidly stubborn / stubbornly stupid      1  edkh6qo
                                                Text  Label       ID
0  Im really sorry about your situation :( Althou...      0  eecwqtt
1   Girlfriend weak as well, that jump was pathetic.      0  eeni74k
2  I've also heard that intriguing but also kinda...      1  edk4e66
3  The thought of shooting anything at asylum see...      1  ed2e00i
4  if the pain doesn't go away after 4 hours or s...      0  eezp1cd


In [ ]:
train_lst_1 = df_train['Text'].tolist()
print(len(train_lst_1))
train_lst_1[:5]
uq_tr_1 = list(set(train_lst_1))
print(len(uq_tr_1))
test_lst_1 = df_test['Text'].tolist()
uq_ts_1 = list(set(test_lst_1))
total_dataset = uq_tr_1 + uq_ts_1
print('Dataset length is', len(total_dataset))

3731
3722
Dataset length is 4393


In [ ]:
# Defining the tokenizer
def get_tokenizer(vocabulary_size):
  print('Training tokenizer...')
  tokenizer = Tokenizer(num_words= vocabulary_size)
  tweet_text = []
  print('Read {} Sentences'.format(len(total_dataset)))
  tokenizer.fit_on_texts(total_dataset)
  return tokenizer

In [ ]:
# For getting the embedding matrix
def get_embeddings():
  print('Generating embeddings matrix...')
  embeddings_file = 'glove.6B.300d.txt'
  embeddings_index = dict()
  with open(embeddings_file, 'r', encoding="utf-8") as infile:
    for line in infile:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      embeddings_index[word] = vector
	# create a weight matrix for words in training docs
  vocabulary_size = len(embeddings_index)
  embeddinds_size = list(embeddings_index.values())[0].shape[0]
  print('Vocabulary = {}, embeddings = {}'.format(vocabulary_size, embeddinds_size))
  tokenizer = get_tokenizer(vocabulary_size)
  embedding_matrix = np.zeros((vocabulary_size, embeddinds_size))
  considered = 0
  total = len(tokenizer.word_index.items())
  for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
      print(word, index)
      continue
    else:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        considered += 1
  print('Considered ', considered, 'Left ', total - considered)			
  return embedding_matrix, tokenizer

In [ ]:
def get_data(tokenizer, MAX_LENGTH, input_df):
  print('Loading data')
  X1, X2, Y = [], [], []
	# with open(input_file) as infile:
	# 	for line in infile:
	# 		data = line.split(',')
	# 		text, annotation = data[2], data[1]
			
	# 		if annotation == "MET":
	# 			X.append(text)
	# 			Y.append("1")
	# 		elif annotation == "Non_MET" or annotation == "Help":	
	# 			X.append(text)
	# 			Y.append("0")
  X1 = input_df['Text'].tolist()
  Y = input_df['Label'].tolist()
  
  assert len(X1) == len(Y)
  sequences_1 = tokenizer.texts_to_sequences(X1)
  X1 = pad_sequences(sequences_1, maxlen=MAX_LENGTH)
  Y = np.array(Y)
  return X1, Y

In [ ]:
embedding_matrix, tokenizer = get_embeddings()

Generating embeddings matrix...
Vocabulary = 400000, embeddings = 300
Training tokenizer...
Read 4393 Sentences
Considered  6733 Left  525


In [ ]:
MAX_LENGTH = 40
# read ml data
X1, Y = get_data(tokenizer, MAX_LENGTH, df_train)

Loading data


In [ ]:
X1_test, Y_test = get_data(tokenizer, MAX_LENGTH, df_test)

Loading data


In [ ]:
from collections import Counter
Counter(Y_test)

Counter({0: 224, 1: 447})

In [ ]:
Counter(Y)

Counter({0: 1887, 1: 1844})

In [ ]:
y_train = keras.utils.to_categorical(Y)
print(y_train)
y_test = keras.utils.to_categorical(Y_test)
print(y_test)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
from sklearn.model_selection import train_test_split
VALIDATION_RATIO = 0.1
RANDOM_STATE = 9527
x1_train, x1_val, \
y_train, y_val = \
    train_test_split(
        X1, y_train, 
        test_size=VALIDATION_RATIO, 
        random_state=RANDOM_STATE
)

In [ ]:
print("Training Set")
print("-" * 10)
print(f"x1_train: {x1_train.shape}")
print(f"y_train : {y_train.shape}")

print("-" * 10)
print(f"x1_val:   {x1_val.shape}")
print(f"y_val :   {y_val.shape}")
print("-" * 10)
print("Test Set")

Training Set
----------
x1_train: (3357, 40)
y_train : (3357, 2)
----------
x1_val:   (374, 40)
y_val :   (374, 2)
----------
Test Set


In [ ]:
NUM_CLASSES = 2

MAX_SEQUENCE_LENGTH = 40

NUM_LSTM_UNITS = 300

MAX_NUM_WORDS = embedding_matrix.shape[0]

NUM_EMBEDDING_DIM = embedding_matrix.shape[1]

In [ ]:
top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')

embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
top_embedded = embedding_layer(
    top_input)

shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)

# merged = concatenate(
#     [top_output, bm_output], 
#     axis=-1)

dense_in =  Dense(
    units=64, 
    activation='tanh')

intermediate = dense_in(top_output)

dense =  Dense(
    units=NUM_CLASSES, 
    activation='softmax')

predictions = dense(intermediate)

model = Model(
    inputs=[top_input], 
    outputs=predictions)

model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 40)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 40, 300)           120000000 
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                19264     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 120,740,594
Trainable params: 120,740,594
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
lr = 1e-3
opt = Adam(lr=lr, decay=lr/50)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='goemotion.h5', verbose=1, save_best_only=True)

In [ ]:
BATCH_SIZE = 128
NUM_EPOCHS = 50
stop = [EarlyStopping(monitor='val_loss', patience=0.001)]
history = model.fit(x=x1_train,
                    y=y_train,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=(
                      x1_val, 
                      y_val
                    ),
                    shuffle=True,
                    callbacks=[checkpointer, stop],
          )

Epoch 1/50
27/27 [==============================] - ETA: 0s - loss: 0.6390 - accuracy: 0.6273
Epoch 00001: val_loss improved from inf to 0.71337, saving model to goemotion.h5
27/27 [==============================] - 38s 1s/step - loss: 0.6390 - accuracy: 0.6273 - val_loss: 0.7134 - val_accuracy: 0.6791
Epoch 2/50
27/27 [==============================] - ETA: 0s - loss: 0.2975 - accuracy: 0.8826
Epoch 00002: val_loss improved from 0.71337 to 0.49420, saving model to goemotion.h5
27/27 [==============================] - 40s 1s/step - loss: 0.2975 - accuracy: 0.8826 - val_loss: 0.4942 - val_accuracy: 0.7647
Epoch 3/50
27/27 [==============================] - ETA: 0s - loss: 0.1127 - accuracy: 0.9628
Epoch 00003: val_loss did not improve from 0.49420
27/27 [==============================] - 31s 1s/step - loss: 0.1127 - accuracy: 0.9628 - val_loss: 0.5703 - val_accuracy: 0.7861


In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
predictions = model.predict(
    [X1_test])
# print('Accuracy is')
# print(metrics.accuracy_score(y_test, y_pred, sample_weight = reduced_test_weights)*100)
# print(classification_report(y_test, y_pred, target_names = ['agreed', 'disagreed'], sample_weight = reduced_test_weights))

In [ ]:
y_pred = [idx for idx in np.argmax(predictions, axis=1)]
#print(y_pred)
print('Accuracy is')
print(metrics.accuracy_score(Y_test, y_pred)*100)
print(classification_report(Y_test, y_pred, target_names = ['true', 'false']))

Accuracy is
78.9865871833085
              precision    recall  f1-score   support

        true       0.63      0.89      0.74       224
       false       0.93      0.74      0.82       447

    accuracy                           0.79       671
   macro avg       0.78      0.82      0.78       671
weighted avg       0.83      0.79      0.80       671



In [ ]:
type(y_pred)
print(len(y_pred))

671
